# Part II — Core Django  
## 15. Email and Notifications (Dev → Production-Grade Patterns)

Email is one of the most common “real-world” requirements in Django projects:

- password reset (already introduced)
- account verification / welcome emails
- alerts (“your article was published”)
- invitations
- billing receipts
- system monitoring notifications
- admin “contact us” submissions

This chapter teaches you email the way it’s done in real teams:
- correct settings per environment
- reliable templated emails (text + HTML)
- link building (absolute URLs)
- safe, testable sending
- production deliverability basics
- notification patterns (idempotency, “don’t spam users”)
- how to keep email logic out of views (service layer)

---

## 15.0 Learning Outcomes

By the end of this chapter, you should be able to:

1. Explain how Django sends email and what an email backend is.
2. Configure email correctly for:
   - development (console/file backend)
   - tests (locmem outbox)
   - production (SMTP/provider)
3. Send emails using:
   - `send_mail`
   - `EmailMessage`
   - `EmailMultiAlternatives` (text + HTML)
4. Build templated emails with `render_to_string` and reusable layouts.
5. Build absolute URLs in emails reliably (with and without `request`).
6. Implement a notification use-case in your app (article published email).
7. Test emails using `django.core.mail.outbox`.
8. Understand production concerns: deliverability, retries, background tasks.

---

## 15.1 Email in Django: The Architecture (What Actually Happens)

### 15.1.1 Django doesn’t “magically send email”
Django provides:
- API to construct messages
- hooks to send through a **backend**
- helpers to render templates

But the actual sending is done by:
- SMTP server you configure, or
- provider API (via third-party package), or
- dev/test backend that doesn’t really deliver

### 15.1.2 Email backend = “where send() goes”
When you call `email.send()`, Django uses `settings.EMAIL_BACKEND`.

Common backends:
- console backend (prints to terminal)
- file backend (writes to disk)
- locmem backend (keeps in memory for tests)
- SMTP backend (real sending)

---

## 15.2 Configure Email Backends by Environment (Industry Standard)

### 15.2.1 Development: console backend
In `config/settings.py` (dev-only):

```python
EMAIL_BACKEND = "django.core.mail.backends.console.EmailBackend"
DEFAULT_FROM_EMAIL = "Django Mastery <no-reply@example.com>"
```

What you get:
- emails print in your runserver terminal
- no external setup
- perfect for learning flows (password reset, publish notifications)

### 15.2.2 Alternative dev backend: file backend (useful for “see HTML nicely”)
```python
EMAIL_BACKEND = "django.core.mail.backends.filebased.EmailBackend"
EMAIL_FILE_PATH = BASE_DIR / "sent_emails"
DEFAULT_FROM_EMAIL = "Django Mastery <no-reply@example.com>"
```

This writes each email to a file. It’s great when you want to open the email body in
an editor and inspect formatting.

### 15.2.3 Tests: locmem backend (Django’s standard)
In tests, Django uses an in-memory backend so you can assert:

- how many emails were sent
- subject/body/to/from
- HTML alternatives

You don’t need to configure it manually for normal tests. You access the outbox via:

```python
from django.core import mail
mail.outbox
```

### 15.2.4 Production: SMTP backend (most common baseline)
In production, you usually set environment variables and configure:

```python
EMAIL_BACKEND = "django.core.mail.backends.smtp.EmailBackend"
EMAIL_HOST = os.environ["EMAIL_HOST"]
EMAIL_PORT = int(os.environ.get("EMAIL_PORT", "587"))
EMAIL_HOST_USER = os.environ.get("EMAIL_HOST_USER", "")
EMAIL_HOST_PASSWORD = os.environ.get("EMAIL_HOST_PASSWORD", "")
EMAIL_USE_TLS = os.environ.get("EMAIL_USE_TLS", "true").lower() == "true"
EMAIL_TIMEOUT = int(os.environ.get("EMAIL_TIMEOUT", "10"))

DEFAULT_FROM_EMAIL = os.environ.get(
    "DEFAULT_FROM_EMAIL",
    "Django Mastery <no-reply@example.com>",
)
```

**Why env vars?**
- credentials must not live in git
- different environments use different servers
- rotating keys/passwords should not require code changes

---

## 15.3 Sending Email: The Three Main APIs (When to Use Which)

### 15.3.1 `send_mail()` (quick and simple)
Good for very simple emails.

```python
from django.core.mail import send_mail
from django.conf import settings

send_mail(
    subject="Welcome",
    message="Thanks for joining.",
    from_email=settings.DEFAULT_FROM_EMAIL,
    recipient_list=["user@example.com"],
)
```

Pros:
- minimal code

Cons:
- less flexible (HTML alternative, attachments, headers are more awkward)

### 15.3.2 `EmailMessage` (more control)
Use when you want:
- headers
- reply-to
- attachments
- explicit control over content type

```python
from django.core.mail import EmailMessage
from django.conf import settings

email = EmailMessage(
    subject="Support request received",
    body="We got your message and will reply soon.",
    from_email=settings.DEFAULT_FROM_EMAIL,
    to=["user@example.com"],
    reply_to=["support@example.com"],
)
email.send()
```

### 15.3.3 `EmailMultiAlternatives` (text + HTML)
This is the professional standard for user-facing emails.

Why you want both:
- some clients prefer plain text
- HTML version improves branding/UX
- deliverability is often better with a text alternative

Example:

```python
from django.core.mail import EmailMultiAlternatives
from django.conf import settings

subject = "Your article was published"
text_body = "Your article is live."
html_body = "<p>Your article is <strong>live</strong>.</p>"

msg = EmailMultiAlternatives(
    subject=subject,
    body=text_body,
    from_email=settings.DEFAULT_FROM_EMAIL,
    to=["user@example.com"],
)
msg.attach_alternative(html_body, "text/html")
msg.send()
```

---

## 15.4 Templated Emails (Industry Standard Approach)

Hardcoding long email bodies in Python becomes unmaintainable quickly. The correct
pattern is:

1. create templates in `templates/emails/...`
2. render them with context
3. send as text + HTML

### 15.4.1 Email template structure
Create:

```text
templates/
  emails/
    base.txt
    base.html
    articles/
      published_subject.txt
      published.txt
      published.html
```

### 15.4.2 Base templates (reuse shared layout)

**`templates/emails/base.txt`**:
```text
{% block body %}{% endblock %}

--
{{ site_name }}
{{ site_url }}
```

**`templates/emails/base.html`** (simple, not fancy):
```django
{% autoescape on %}
<!doctype html>
<html>
  <body style="font-family: Arial, sans-serif;">
    <div style="max-width: 640px; margin: 0 auto;">
      {% block body %}{% endblock %}

      <hr />
      <p style="color: #666; font-size: 12px;">
        {{ site_name }} • <a href="{{ site_url }}">{{ site_url }}</a>
      </p>
    </div>
  </body>
</html>
{% endautoescape %}
```

**Why base templates matter**
- consistent signature/footer
- consistent styling for HTML emails
- changes in one place affect all emails

---

## 15.5 Build a Real Notification: “Article Published” Email

We’ll implement a notification that fires when an article transitions to `PUBLISHED`.

### 15.5.1 Why “status transition” matters (don’t spam)
If you send “published” email on every save, you’ll spam users.

Correct behavior:
- send only when:
  - old status was not published, and
  - new status is published

This is an example of **idempotency** in notifications: the same action should not
send duplicates.

---

## 15.6 Add Site URL Settings (Needed for Email Links)

Email must include absolute URLs like:

```text
https://example.com/articles/html/hello-django/
```

If you only use relative paths (`/articles/...`), the email client doesn’t know the
domain.

### 15.6.1 Add `SITE_NAME` and `SITE_URL` to settings (env-based)
In `config/settings.py`:

```python
import os

SITE_NAME = os.environ.get("SITE_NAME", "Django Mastery Workbook")
SITE_URL = os.environ.get("SITE_URL", "http://127.0.0.1:8000")
```

In production:
- set `SITE_URL=https://yourdomain.com`

---

## 15.7 Write Email Templates for “Article Published”

### 15.7.1 Subject template
**`templates/emails/articles/published_subject.txt`**
```text
Your article is published: {{ article.title }}
```

Subject templates should be single-line. Django template rendering often includes a
trailing newline—your sending code should `.strip()` it.

### 15.7.2 Text body
**`templates/emails/articles/published.txt`**
```text
{% extends "emails/base.txt" %}

{% block body %}
Hi {{ user.username }},

Your article has been published.

Title: {{ article.title }}
Link: {{ article_url }}

If you did not expect this, contact support.

{% endblock %}
```

### 15.7.3 HTML body
**`templates/emails/articles/published.html`**
```django
{% extends "emails/base.html" %}

{% block body %}
  <p>Hi {{ user.username }},</p>

  <p>Your article has been <strong>published</strong>.</p>

  <p>
    <strong>{{ article.title }}</strong><br />
    <a href="{{ article_url }}">{{ article_url }}</a>
  </p>

  <p>If you did not expect this, contact support.</p>
{% endblock %}
```

---

## 15.8 Implement an Email Service Function (Keep Views Clean)

Professional rule:
- views should orchestrate
- email composition/sending should live in a service module

Create `articles/services.py` (or a package `articles/services/notifications.py`).
We’ll keep it simple as `services.py`.

**`articles/services.py`**
```python
from django.conf import settings
from django.core.mail import EmailMultiAlternatives
from django.template.loader import render_to_string
from django.urls import reverse


def build_absolute_uri(path: str) -> str:
    base = settings.SITE_URL.rstrip("/")
    return f"{base}{path}"


def send_article_published_email(*, article) -> None:
    """
    Sends a text+HTML email to the article author when an article is published.

    This function assumes:
    - article has an author with an email address.
    - SITE_URL is set correctly for the environment.
    """
    user = article.author
    if not user.email:
        return  # Decide your policy: skip, raise, or notify admins.

    article_path = reverse("articles:detail_html", kwargs={"slug": article.slug})
    article_url = build_absolute_uri(article_path)

    context = {
        "site_name": getattr(settings, "SITE_NAME", "Site"),
        "site_url": getattr(settings, "SITE_URL", ""),
        "user": user,
        "article": article,
        "article_url": article_url,
    }

    subject = render_to_string(
        "emails/articles/published_subject.txt",
        context,
    ).strip()

    text_body = render_to_string("emails/articles/published.txt", context)
    html_body = render_to_string("emails/articles/published.html", context)

    msg = EmailMultiAlternatives(
        subject=subject,
        body=text_body,
        from_email=settings.DEFAULT_FROM_EMAIL,
        to=[user.email],
    )
    msg.attach_alternative(html_body, "text/html")
    msg.send()
```

### 15.8.1 Why this service layer is the right design
- keeps email logic reusable (admin action, signal, API endpoint can call it)
- makes unit testing easier
- reduces view complexity
- isolates future changes (switch provider, add tracking headers, etc.)

### 15.8.2 Why `SITE_URL` instead of `request.build_absolute_uri`
If you send emails from background tasks later (Celery), you won’t have a request.
`SITE_URL` works everywhere.

You can still use `request.build_absolute_uri()` when you *do* have request, but
service functions often shouldn’t depend on request objects.

---

## 15.9 Trigger the Email Only on Publish Transition

We’ll implement this in the **edit view** (and also in create). The key is to detect
the status before save.

### 15.9.1 Update create view
In `articles/views.py` (inside valid POST for create), after saving:

```python
from .services import send_article_published_email
from .models import Article

# inside article_create POST success
article = form.save(commit=False)
article.author = request.user
old_status = None
article.save()
form.save_m2m()

if article.status == Article.Status.PUBLISHED:
    send_article_published_email(article=article)
```

For create, “transition” is basically “is it published on creation?”

### 15.9.2 Update edit view with transition check
In `article_edit`, capture old status first:

```python
from .services import send_article_published_email

# inside article_edit
old_status = article.status

# after form.is_valid()
article = form.save()

published_now = (
    old_status != Article.Status.PUBLISHED
    and article.status == Article.Status.PUBLISHED
)

if published_now:
    send_article_published_email(article=article)
```

### 15.9.3 Why we do it this way
- prevents duplicate “published” emails
- works even if user edits body multiple times after publishing
- keeps logic explicit and easy to test

---

## 15.10 Add a Simple “Contact Us” Email (Forms + Email Together)

This is a very common pattern and reinforces everything.

### 15.10.1 Form (if you already created ContactForm earlier, reuse it)
`pages/forms.py`:

```python
from django import forms


class ContactForm(forms.Form):
    name = forms.CharField(max_length=100)
    email = forms.EmailField()
    message = forms.CharField(min_length=10, widget=forms.Textarea)
```

### 15.10.2 View that sends an email
`pages/views.py`:

```python
from django.conf import settings
from django.contrib import messages
from django.core.mail import send_mail
from django.shortcuts import redirect, render
from django.views.decorators.http import require_http_methods

from .forms import ContactForm


@require_http_methods(["GET", "POST"])
def contact(request):
    if request.method == "POST":
        form = ContactForm(request.POST)
        if form.is_valid():
            name = form.cleaned_data["name"]
            email = form.cleaned_data["email"]
            message = form.cleaned_data["message"]

            subject = f"Contact form: {name}"
            body = (
                f"From: {name} <{email}>\n\n"
                f"Message:\n{message}\n"
            )

            send_mail(
                subject=subject,
                message=body,
                from_email=settings.DEFAULT_FROM_EMAIL,
                recipient_list=["support@example.com"],
                reply_to=[email],
            )
            messages.success(request, "Thanks! We received your message.")
            return redirect("pages:contact")

    else:
        form = ContactForm()

    return render(request, "pages/contact.html", {"form": form})
```

Template `pages/templates/pages/contact.html`:

```django
{% extends "base.html" %}

{% block title %}Contact{% endblock %}

{% block content %}
  <h1>Contact</h1>

  <form method="post">
    {% csrf_token %}

    <div class="field">
      {{ form.name.label_tag }} {{ form.name }}
      {% for err in form.name.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      {{ form.email.label_tag }} {{ form.email }}
      {% for err in form.email.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      {{ form.message.label_tag }} {{ form.message }}
      {% for err in form.message.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <button type="submit">Send</button>
  </form>
{% endblock %}
```

URL `pages/urls.py`:

```python
path("contact/", views.contact, name="contact"),
```

With console backend, submitting the form prints the email in your terminal.

---

## 15.11 Testing Emails (The Correct, Professional Way)

### 15.11.1 Understand `mail.outbox`
In Django tests, sent emails go to an in-memory list: `mail.outbox`.

This makes email tests:
- fast
- deterministic
- no external services

### 15.11.2 Test: “published email sent on transition”
Create `articles/tests_email.py`:

```python
from django.contrib.auth import get_user_model
from django.core import mail
from django.test import TestCase
from django.urls import reverse
from django.utils import timezone

from articles.models import Article


class ArticleEmailTests(TestCase):
    def setUp(self):
        User = get_user_model()
        self.user = User.objects.create_user(
            username="u1",
            password="pass12345",
            email="u1@example.com",
        )
        self.client.login(username="u1", password="pass12345")

        self.article = Article.objects.create(
            title="Draft",
            slug="draft",
            body="Body content long enough.",
            status=Article.Status.DRAFT,
            author=self.user,
        )

    def test_publish_transition_sends_email_once(self):
        url = reverse("articles:edit", kwargs={"slug": "draft"})

        # Transition draft -> published
        response = self.client.post(
            url,
            {
                "title": "Draft",
                "slug": "draft",
                "body": "Body content long enough.",
                "status": Article.Status.PUBLISHED,
                "published_at": timezone.now(),
                "tags": [],
            },
        )
        self.assertEqual(response.status_code, 302)
        self.assertEqual(len(mail.outbox), 1)

        email = mail.outbox[0]
        self.assertIn("published", email.subject.lower())
        self.assertEqual(email.to, ["u1@example.com"])

        # Editing again while already published should not send another publish email
        response2 = self.client.post(
            url,
            {
                "title": "Draft updated",
                "slug": "draft",
                "body": "Body content long enough.",
                "status": Article.Status.PUBLISHED,
                "published_at": timezone.now(),
                "tags": [],
            },
        )
        self.assertEqual(response2.status_code, 302)
        self.assertEqual(len(mail.outbox), 1)
```

### 15.11.3 Test: contact form sends an email
Create `pages/tests_email.py`:

```python
from django.core import mail
from django.test import TestCase
from django.urls import reverse


class ContactEmailTests(TestCase):
    def test_contact_form_sends_email(self):
        url = reverse("pages:contact")
        response = self.client.post(
            url,
            {
                "name": "Asha",
                "email": "asha@example.com",
                "message": "Hello, I need help with my account.",
            },
            follow=True,
        )
        self.assertEqual(response.status_code, 200)
        self.assertEqual(len(mail.outbox), 1)
        self.assertIn("Contact form", mail.outbox[0].subject)
```

---

## 15.12 Production Deliverability Basics (You Need These Even as a Backend Dev)

Even perfect code can “send” emails that never arrive. Deliverability depends on:

1. **SPF**: authorizes which servers can send mail for your domain
2. **DKIM**: cryptographically signs email to prove authenticity
3. **DMARC**: policy + reporting combining SPF/DKIM results
4. **From address discipline**:
   - sending from `no-reply@yourdomain.com` with proper DNS setup
   - avoid random from addresses that don’t match your domain
5. **Content**:
   - avoid spammy phrases/links
   - include text alternative
6. **List hygiene**:
   - handle bounces/complaints
   - don’t keep sending to dead addresses

In many companies, you’ll use transactional providers (SendGrid/Mailgun/Postmark/AWS
SES). Django can talk to them via SMTP or provider-specific APIs (with packages).

---

## 15.13 Notifications: Patterns That Prevent Future Pain

### 15.13.1 Transactional vs marketing (separate streams)
- Transactional: password reset, verification, receipts (must deliver)
- Marketing: newsletters (must support opt-out, compliance)

Don’t mix them:
- different sender addresses/domains
- different compliance requirements
- different user expectations

### 15.13.2 Idempotency (don’t double-send)
You already implemented a publish transition check. More advanced patterns include:
- storing “notification sent” timestamps in DB
- unique constraints on “event id + user + type”
- task queues with deduplication keys

### 15.13.3 Don’t block request/response on slow email in production
SMTP/provider calls can be slow or fail.

Industry pattern:
- enqueue email sending to a background worker (Celery/RQ)
- retry on failures
- keep request fast

We’ll implement background jobs later; for now, understand:
- sending email inside a view is acceptable for learning
- but not ideal for high-traffic production

---

## 15.14 Exercises (Do These Before Proceeding)

1. Add “Email me when my article is published” preference:
   - create a simple model `NotificationPreference` with OneToOne to User
   - boolean `email_on_publish`
   - default True
   - update `send_article_published_email` to respect the preference

2. Add an email template for “Article updated” (but only send once per day per
   user—design a strategy even if you don’t fully implement it).

3. Switch dev backend from console to file backend:
   - send a contact email
   - open the saved file and confirm the full email content

4. Add a unit test that asserts the HTML alternative exists:
   - `email.alternatives` contains one tuple with content type `text/html`

---

## 15.15 Chapter Summary

- Django sends email through configurable backends; dev/test/prod should differ.
- The professional default is **templated text + HTML** via `EmailMultiAlternatives`.
- Email logic belongs in a service layer, not inside views.
- Email links must be absolute; use `SITE_URL` (works even in background tasks).
- Test email sending using `mail.outbox`.
- Production email requires deliverability configuration (SPF/DKIM/DMARC) and often
  background sending + retries.

---

Next chapter: **Part III — Project 1 (Beginner): Blog/Content Site** (Chapter 16)  
We’ll consolidate everything you’ve built into a coherent mini product: requirements,
data model, public pages, admin workflow, search/pagination, and a production-style
checklist.